# Data Aggregation and Group Operations

In [1]:
# Categorizing a data set and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow.
# After loading, merging, and preparing a dataset, you may need to compute group statistics or possible pivot tables for reporting or visualization purpose.
# pandas provides a flexible and high performance groupby facility, enabling you to slice and dice, and summarize data sets in a natural way.

# GroupBy mechanics

In [2]:
import pandas as pd 
import numpy as np

df = pd.DataFrame({ 'key1' : ['a', 'a', 'b', 'b', 'a'],
                    'key2' : ['one', 'two', 'one', 'two', 'one'],
                    'data1' : np.random.randn(5),
                    'data2' : np.random.randn(5)}) 

In [3]:
df.head()

,key1,key2,data1,data2
0,a,one,1.323361,-0.353553
1,a,two,-0.763190,2.185220
2,b,one,-0.049697,0.494983
3,b,two,0.977056,1.473394
4,a,one,-0.720139,-0.071264


In [6]:
# Suppose you wanted to compute the mean of the data1 column using the labels from key1. 
# There are a number of ways to do this. One is to access data1 and call groupby with the column(a Series) at key1: 
grouped = df['data1'].groupby(df['key1']) # group data1 by key1
grouped # returns a GroupBy object 

In [8]:
# # This "grouped" variable is now a GroupBy object. It has not actualy computed anything yet except for some intermediate data about the group key df['key1'].
# # The idea is that this object has all of the information needed to then apply some operation to each of the groups. 
# # For example, to compute group means we can call the GroupBy's mean method: 
grouped.mean()

key1
a   -0.053323
b    0.463680
Name: data1, dtype: float64

In [10]:
# If instead we had passed multiple arrays as a list, we'd get something different: 

means = df['data1'].groupby([df['key1'], df['key2']]).mean() # group data1 by key1 and key2 and compute mean 
means

key1  key2
a     one     0.301611
      two    -0.763190
b     one    -0.049697
      two     0.977056
Name: data1, dtype: float64

In [12]:
# # In this case, we grouped the data using two keys, and the resulting Series now has a hierarchical index consisting of  the unique pairs of keys observed:
means.unstack() # unstack the data 

key2,one,two
key1,,
a,0.301611,-0.763190
b,-0.049697,0.977056


In [14]:
# In these examples, the group keys are all Series, though they could be any arrays of the right length:

states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio']) 
years = np.array([2005, 2005, 2006, 2005, 2006])

In [16]:
df['data1'].groupby([states, years]).mean() # group data1 by states and years and compute mean 

California  2005   -0.763190
            2006   -0.049697
Ohio        2005    1.150208
            2006   -0.720139
Name: data1, dtype: float64

In [20]:
# Frequently the grouping information is found in the same DataFrame as the data you want to work on. 
# In that case, you can pass column names(whther those are strings, numbers, or other Python objects) as the group keys:
df.groupby('key1').mean() # group df by key1 and compute mean 

/var/folders/vq/73zp4qns2cjdsxcw0bzvh98w0000gn/T/ipykernel_98921/3017464506.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('key1').mean() # group df by key1 and compute mean


,data1,data2
key1,,
a,-0.235384,0.006903
b,-0.148516,-0.131355


In [23]:
df.groupby(['key1', 'key2']).mean() 
# # You may have noticed in the first case df.groupby('key1').mean() that there is no key2 column in the result.
# # Because df['key2'] is not numeric data, it is said to be a nuisance column, which is therefore excluded from the result.
# # By default, all of the numeric columns are aggregated, though it is possible to filter down to a subset, as you'll see soon.

data1     data2
key1 key2                    
a    one   0.195111  0.030563
     two  -1.096372 -0.040417
b    one  -0.070876  0.416844
     two  -0.226155 -0.679555

In [24]:
# Regardless of the objective in using groupby, a generally useful GroupBy method is size, which returns a Series containing group sizes:
df.groupby(['key1', 'key2']).size() # group df by key1 and key2 and compute size 

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

# Iterating Over Groups

In [25]:
# The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data: 
# Consider the following:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.249953 -0.323921
1    a  two -1.096372 -0.040417
4    a  one  0.640174  0.385047
b
  key1 key2     data1     data2
2    b  one -0.070876  0.416844
3    b  two -0.226155 -0.679555


In [26]:
# In the case of multiple keys, the first element in the tuple will be a tuple of key values: 
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2) 
    print(group) 

a one
  key1 key2     data1     data2
0    a  one -0.249953 -0.323921
4    a  one  0.640174  0.385047
a two
  key1 key2     data1     data2
1    a  two -1.096372 -0.040417
b one
  key1 key2     data1     data2
2    b  one -0.070876  0.416844
b two
  key1 key2     data1     data2
3    b  two -0.226155 -0.679555


In [27]:
# Of course you can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of teh data pieces as a one-liner:

pieces = dict(list(df.groupby('key1'))) # group df by key1 and store the data in a dict

In [28]:
pieces['b'] # print the data for key1 = b 

,key1,key2,data1,data2
2,b,one,-0.070876,0.416844
3,b,two,-0.226155,-0.679555


In [29]:
# By default groupby groups on axis=0, but you can group on any of the other axes. For example, we could group the columns of our example df here by dtype like so:
df.dtypes 

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [30]:
grouped = df.groupby(df.dtypes, axis=1) 

In [31]:
dict(list(grouped)) # group df by dtypes and store the data in a dict

{dtype('float64'):       data1     data2
 0 -0.249953 -0.323921
 1 -1.096372 -0.040417
 2 -0.070876  0.416844
 3 -0.226155 -0.679555
 4  0.640174  0.385047,
 dtype('O'):   key1 key2
 0    a  one
 1    a  two
 2    b  one
 3    b  two
 4    a  one}

# Selecting a column or Subset of Columns

In [19]:
# Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of 
# selecting those columns for aggregation. This means that: 
df.groupby('key1')['data1'] # group df by key1 and select data1 column
df.groupby('key1')[['data2']] # group df by key1 and select data2 column
# are syntactic sugar for: 
df['data1'].groupby(df['key1']) # group data1 by key1
df[['data2']].groupby(df['key1']) # group data2 by key1 

In [21]:
# Especially for large datasets, it may be desirable to aggregate only a few columns. 
# For example, in the preceding dataset, to compute means for just the data2 column and get the result as a DataFrame, we could write:
df.groupby(['key1', 'key2'])[['data2']].mean() # group df by key1 and key2 and compute mean of data2 column 

data2
key1 key2          
a    one  -0.212408
     two   2.185220
b    one   0.494983
     two   1.473394

In [22]:
# The object returned by this indexing operation is a grouped DataFrame if a list or array is passed and a grouped Series 
# is just a single column name is passed as a scalar: 
s_grouped = df.groupby(['key1', 'key2'])['data2'] 

In [23]:
s_grouped.mean() # group df by key1 and key2 and compute mean of data2 column 

key1  key2
a     one    -0.212408
      two     2.185220
b     one     0.494983
      two     1.473394
Name: data2, dtype: float64

# Grouping with Dicts and Series 

In [24]:
# Grouping information may exist in a form other than an array. Let's consider another example DataFrame:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis']) 

In [25]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values 

In [26]:
people

,a,b,c,d,e
Joe,1.401812,-0.178154,1.538714,0.296120,0.368429
Steve,-0.165176,-0.515568,0.927953,-0.571039,0.310042
Wes,0.163607,NaN,NaN,-1.197289,0.700270
Jim,-1.500130,-0.788464,1.619117,-1.642483,-1.059542
Travis,1.587649,0.708425,0.305256,1.141281,0.578276


In [27]:
# Now suppose I have a group correspondence for the columns and want to sum together the columns by group:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [28]:
# Now you could easily construct an array from this dict to pass to groupby, but instead we can just pass the dict: 
by_column = people.groupby(mapping, axis=1) # group people by mapping and axis = 1

In [29]:
by_column.sum() # group people by mapping and axis = 1 and compute sum 

,blue,red
Joe,1.834834,1.592088
Steve,0.356915,-0.370703
Wes,-1.197289,0.863878
Jim,-0.023367,-3.348135
Travis,1.446537,2.874350


In [30]:
# The same functionality holds for Series, which can be viewwed as a fixed size mapping. 
# When I used Series as group keys in the above examples, pandas does in fact, inspect each Series to ensure that 
# its index is aligned with the axis it's grouping:
map_series = pd.Series(mapping) # create a Series from mapping 

In [31]:
map_series 

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
people.groupby(map_series, axis=1).count()  # group people by map_series and axis = 1 and compute count

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


# Grouping with Functions

In [33]:
# Using Python functions in what can be fairly creative ways is a more abstract way of defining a group mapping compared with a dict or Series. 
# Any function passed as a group key will be called once per index value, with the return values being used as the group names.
# More concretely, consider the example DataFrame from the previous section, which has people's first names as index values.
# Suppose you wanted to group by the length of the names; you could compute an array of string lengths, but instead you you can just pass the "len" function: 
people.groupby(len).sum() # group people by len and compute sum 

,a,b,c,d,e
3,0.065290,-0.966618,3.157831,-2.543652,0.009157
5,-0.165176,-0.515568,0.927953,-0.571039,0.310042
6,1.587649,0.708425,0.305256,1.141281,0.578276


In [34]:
# Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:
key_list = ['one','one', 'two','two','two']

In [35]:
people.groupby([len, key_list]).min() # group people by len and key_list and compute min 

a         b         c         d         e
3 one  1.401812 -0.178154  1.538714  0.296120  0.368429
  two -1.500130 -0.788464  1.619117 -1.642483 -1.059542
5 one -0.165176 -0.515568  0.927953 -0.571039  0.310042
6 two  1.587649  0.708425  0.305256  1.141281  0.578276

# Grouping by Index Levels

In [36]:
# A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index. 
# To do this, pass the level number or name using the level keyword:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])

In [37]:
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['city', 'tenor'])

In [38]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns) # create a DataFrame from columns 

In [39]:
hier_df 

city         US                            JP          
tenor         1         3         5         1         3
0      0.600929  0.623682  0.931289  0.132884 -0.051356
1     -1.325800 -0.514190 -0.964203 -0.981152 -0.605048
2     -0.311408  0.434293  1.186743 -0.386246  0.649426
3      1.382170 -0.967929 -1.049034 -1.214956 -0.701267

In [40]:
hier_df.groupby(level='city', axis=1).count() # group hier_df by level = 'city' and axis = 1 and compute count 

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Data Aggregation

In [41]:
# By aggregation, I am referring to any data transformation that produces scalar values from arrays. 
# In the examples above I have used several of them, including mean, count, min, and sum.
# You may wonder what is going on when you invoke mean() on a GroupBy object.
# Many common aggregations, such as those found above, have optimized implementations. 
# However, you are not limited to only this set of methods. 
# You can use aggregations of your own devising and additionally call any method that is also defined on the grouped object.
# For example, you might recall that quantile computes sample quantiles of a Series or a DataFrame's columns.

df

,key1,key2,data1,data2
0,a,one,1.323361,-0.353553
1,a,two,-0.763190,2.185220
2,b,one,-0.049697,0.494983
3,b,two,0.977056,1.473394
4,a,one,-0.720139,-0.071264


In [42]:
grouped = df.groupby('key1') # group df by key1 

In [44]:
grouped['data1'].quantile(0.9) # group data1 by key1 and compute quantile 

key1
a    0.914661
b    0.874380
Name: data1, dtype: float64

In [45]:
# While quantile is not explicitly implemented for GroupBy, it is a Series method and thus available for use.
# Internally, GroupBy efficiently slices up the Series, calls piece.quantile(0.9) for each piece, and then 
# assembles those results together into the result object: 

# To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method:

def peak_to_peak(arr):
    return arr.max() - arr.min() 

In [46]:
grouped.agg(peak_to_peak)   # group df by key1 and compute peak_to_peak 

/var/folders/vq/73zp4qns2cjdsxcw0bzvh98w0000gn/T/ipykernel_13770/3668518599.py:1: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)   # group df by key1 and compute peak_to_peak


,data1,data2
key1,,
a,2.086551,2.538773
b,1.026752,0.978411


In [48]:
# You will notice that some methods like  "describe"  also work, even though they are not aggregations, 
# strictly speaking: 
grouped.describe().T  # group df by key1 and compute describe 

key1                a         b
data1 count  3.000000  2.000000
      mean  -0.053323  0.463680
      std    1.192437  0.726023
      min   -0.763190 -0.049697
      25%   -0.741665  0.206991
      50%   -0.720139  0.463680
      75%    0.301611  0.720368
      max    1.323361  0.977056
data2 count  3.000000  2.000000
      mean   0.586801  0.984188
      std    1.391448  0.691841
      min   -0.353553  0.494983
      25%   -0.212408  0.739586
      50%   -0.071264  0.984188
      75%    1.056978  1.228791
      max    2.185220  1.473394

In [49]:
# Optimized groupby methods 
# Function name       Description 
# count               Number of non-NA values in the group
# sum                 Sum of non-NA values
# mean                Mean of non-NA values
# median              Arithmetic median of non-NA values
# std, var            Unbiased (n-1 denominator) standard deviation and variance
# min, max            Minimum and maximum of non-NA values
# prod                Product of non-NA values
# first, last         First and last non-NA values

# Column-Wise and Multiple Function Application

In [50]:
# As you've seen above, aggregating a Series or all of the columns of a DataFrame is a matter of using aggregate with the desired function 
# or calling a method like mean or std. 
# However, you may want to aggregate using a different function depending on the column, or multiple functions at once.
# Fortunately, this is possible to do, which I'll illustrate through a number of examples. 
grouped = df.groupby('key1') # group df by key1 

In [51]:
grouped_pct = grouped['data1']

In [52]:
grouped_pct.agg('mean') 

key1
a   -0.053323
b    0.463680
Name: data1, dtype: float64

In [53]:
grouped_pct.agg(['mean', 'std', peak_to_peak]) # group grouped_pct by mean, std, peak_to_peak 

,mean,std,peak_to_peak
key1,,,
a,-0.053323,1.192437,2.086551
b,0.463680,0.726023,1.026752


In [54]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) # group grouped_pct by mean and std 

,foo,bar
key1,,
a,-0.053323,1.192437
b,0.463680,0.726023


In [55]:
# You don't need to accept the names that GroupBy gives to the columns; notably lambda functions have the name '<lambda>', 
# which makes them hard to identify (you can see for yourself by looking at a function's __name__ attribute).
# Thus, if you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names
# (you can think of a list of 2-tuples as an ordered mapping):
functions = ['count', 'mean', 'max'] 

In [56]:
result = grouped['data1'].agg(functions) # group grouped by data1 and compute functions 

In [57]:
result 

,count,mean,max
key1,,,
a,3,-0.053323,1.323361
b,2,0.463680,0.977056


In [59]:
result['count'] # print the count column

key1
a    3
b    2
Name: count, dtype: int64

# Group - wise Operations and Transformations

In [60]:
# Aggregation is only one kind of data transformation. It is a special case in the more general class of data transformations.
# that is, it accepts functions that reduce a one-dimensional array to a scalar value.
# I will introduce the "transform" and "apply" methods, which will enable you to do many other kinds of group operations. 

# Suppose, instead, we wanted to add a column to a DataFrame containing group means for each index.
# One way to do this to aggregate, then merge:
df

,key1,key2,data1,data2
0,a,one,1.323361,-0.353553
1,a,two,-0.763190,2.185220
2,b,one,-0.049697,0.494983
3,b,two,0.977056,1.473394
4,a,one,-0.720139,-0.071264


In [61]:
k1_means = df.groupby('key1').mean().add_prefix('mean_') # group df by key1 and compute mean and add prefix 'mean_' 

/var/folders/vq/73zp4qns2cjdsxcw0bzvh98w0000gn/T/ipykernel_13770/1545172360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  k1_means = df.groupby('key1').mean().add_prefix('mean_') # group df by key1 and compute mean and add prefix 'mean_'


In [62]:
k1_means

,mean_data1,mean_data2
key1,,
a,-0.053323,0.586801
b,0.463680,0.984188


In [63]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,1.323361,-0.353553,-0.053323,0.586801
1,a,two,-0.763190,2.185220,-0.053323,0.586801
4,a,one,-0.720139,-0.071264,-0.053323,0.586801
2,b,one,-0.049697,0.494983,0.463680,0.984188
3,b,two,0.977056,1.473394,0.463680,0.984188


In [64]:
# This works, but is somewhat inflexible. 
# You can think of the operation as transforming the two data columns using the mean function, which is called a transform.
# Let's look back at the people DataFrame, which has multiple groups. 

key = ['one', 'two', 'one', 'two', 'one'] 

In [65]:
people.groupby(key).mean() # group people by key and compute mean 

,a,b,c,d,e
one,1.051023,0.265136,0.921985,0.080038,0.548992
two,-0.832653,-0.652016,1.273535,-1.106761,-0.374750


In [66]:
people.groupby(key).transform(np.mean) # group people by key and compute mean 

,a,b,c,d,e
Joe,1.051023,0.265136,0.921985,0.080038,0.548992
Steve,-0.832653,-0.652016,1.273535,-1.106761,-0.374750
Wes,1.051023,0.265136,0.921985,0.080038,0.548992
Jim,-0.832653,-0.652016,1.273535,-1.106761,-0.374750
Travis,1.051023,0.265136,0.921985,0.080038,0.548992


In [67]:
# As you may guess, "transform" applies a function to each group, then places the results in the appropriate locations.
# If each group produces a scalar value, it will be propagated (broadcasted). 
# Suppose instead you wanted to subtract the mean value from each group.
# To do this, create a demeaning function and pass it to transform:
def demean(arr):
    return arr - arr.mean() 

In [68]:
demeaned = people.groupby(key).transform(demean) # group people by key and compute demean 

In [69]:
demeaned

,a,b,c,d,e
Joe,0.350789,-0.443290,0.616729,0.216082,-0.180563
Steve,0.667477,0.136448,-0.345582,0.535722,0.684792
Wes,-0.887415,NaN,NaN,-1.277326,0.151278
Jim,-0.667477,-0.136448,0.345582,-0.535722,-0.684792
Travis,0.536626,0.443290,-0.616729,1.061244,0.029284


# Apply: General split-apply-combine 

In [71]:
# Like "aggregate", "transform" is a more specialized funciton having rigid requirements: 
# The function passed to transform must either produce a scalar value to be broadcasted (like np.mean) or a transformed array of the same size. 
# The most general purpose GroupBy method is "apply".
# Which is the subject of the rest of this section. 
# "apply" splits the object being manipulated into pieces, invokes the passed function on each piece, then attempts to concatenate the pieces together. 
# Returning to the tipping data set above, suppose you wanted to select the top five values by group. 

# First, write a function that selects the rows with the largest values in a particular column: 
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]   # sort df by column and select the last n rows 

In [73]:
# top(tips, n=6) # select the last 6 rows of tips 

In [74]:
# Now if we group by smoker, say, and call apply with this function, we get the following: 
# tips.groupby('smoker').apply(top) # group tips by smoker and apply top function 